## Проект Megafon
**Шаг 1: Загрузка и первичная обработка данных**

## Задача:
построить  алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.
## Данные:
В качестве исходных данных вам будет доступна информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.
Отдельным набором данных будет являться нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.
Данные train и test разбиты по периодам – на train доступно 4 месяцев, а на test отложен последующий месяц. 

**Входные данные:**

    * data_train.csv: id, vas_id, buy_time, target
    
    * features.csv.zip: id, <feature_list> 

И тестовый набор:

    * data_test.csv: id, vas_id, buy_time
    
**target** - целевая переменная, где 1 означает подключение услуги, 0 - абонент не подключил услугу соответственно.

**buy_time** - время покупки, представлено в формате timestamp, для работы с этим столбцом понадобится функция datetime.fromtimestamp из модуля datetime.

**id** - идентификатор абонента

**vas_id** - подключаемая услуга

## Метрика
Скоринг будет осуществляться функцией f1, невзвешенным образом, как например делает функция sklearn.metrics.f1_score(…, average=’macro’).
 sklearn.metrics.f1_score — scikit-learn 0.22.1 documentation


## Формат представления результата
    1. Работающая модель в формате pickle, которая принимает файл data_test.csv из корневой папки и записывает в эту же папку файл answers_test.csv. В этом файле должны находится 4 столбца: buy_time, id, vas_id и target. Target можно записать как вероятность подключения услуги.
    2. Код модели можно представить в виде jupyter-ноутбука. 
    3. Презентация в формате .pdf, в которой необходимо отразить:
        ◦ Информация о модели, ее параметрах, особенностях и основных результатах.
        ◦ Обоснование выбора модели и ее сравнение с альтернативами.
        ◦ Принцип составления индивидуальных предложений для выбранных абонентов.
Рекомендуемое количество слайдов – 5 – 10.
Файл answers_test.csv с результатами работы модели, презентацию, ноутбуки и резюме необходимо прикрепить ко второму уроку “курсовой проект”.

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', None)

from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.2.


In [4]:
#import dask.dataframe as dd не стала использовать dask, так как слияние удобнее и быстрее сделать в pandas

### 1.1 Данные

In [5]:
# сразу уберем Unnamed, так как это те же индексы
# признаки анонимизированные, так что мы не знаем, что они содержат
features = pd.read_csv('features.csv',sep='\t').drop(columns=['Unnamed: 0'])
features.head()

,id,buy_time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,-65.076097,-6.783660,-30.006538,-2.736081,-4.007526,-2.558912,49.520873,38.19189,-0.000725,-0.016435,-0.107041,-1.17746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-42.026959,-2841.496068,-1085.821501,-1755.674564,-89.504287,-119.724355,-70.712019,-54.069191,-16.642826,-7.896282,-5.634035,-10.717958,-28.571103,57.869716,52.911014,26.828289,26.668705,-4.958702,38.254749,-1.671324,-0.001656,1.318354,2.117335,-0.265234,0.331838,0.078356,-0.237576,0.254338,-0.028454,-0.044465,3.698872,26.411990,-0.036834,3.869969,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,4.027863,-11.955314,-1.019293,-0.473446,-2.62084,-1087.017387,-1757.811263,-0.36799,0.396143,-2844.828651,-2298.725139,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,-0.058077,0.129549,0.274871,-3.618164,-11.681641,-0.573283,0.531557,0.582717,-190.670372,1.856777,3.277409,2.174027,4.064012,0.0,-1.276187,-0.020137,-0.042636,-29.797016,-70.470802,-14.96363,-34.888325,-3.861461,-0.317164,-0.007024,-0.143269,60.582329,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,0.280492,-21.183166,-44.376426,-25.320085,-51.984826,-23.961228,-54.128903,-11.614497,-30.288386,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-9.978121,-16.684052,4.645192,13.112964,-0.034569,-0.163184,-109.036398,0.533317,-1.929048,0.376263,-0.228106,-0.251959,-0.000567,0.566264,-0.000708,-0.02921,0.895335,-0.001358,0.0,0.039208,0.665644,-0.008999,-11953.712824,-45175.257711,0.377099,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,9.595373e+08,-42.014078,-4.405604e+08,1.356414e+09,5.565998,-1.465191,-33.302382,-249.128986,-36.772492,-0.364694,-0.133771,-0.209468,-32.356505,-109.884564,-876.69102,-5.368281,-247.110707,-108.409742,-512.437331,-84.617978,-17.295406,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,-53.216097,-6.783660,-26.544905,-2.736081,-4.007526,-2.558912,67.300873,55.97189,-0.000725,-0.016435,-0.107041,15.77254,-3.178521,411.379185,-10.744164,-0.094251,-0.001733,-0.009327,131.407791,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,-2937.657200,-1181.982633,-1755.674564,447.193953,1258.981645,-119.662019,-54.602524,-65.059494,2

In [27]:
# посмотрим, есть ли дубли в id - есть
# но это нормально, так как у нас пары : юзер-услуга
id_double = features.id.value_counts()
id_double[id_double > 1] 

2456449    2
868170     2
4350738    2
2837963    2
2038831    2
          ..
2588556    2
3307448    2
1707210    2
3533762    2
1030284    2
Name: id, Length: 149789, dtype: int64

In [29]:
# посмотрим пару для примера - видим, что у одного юзера может быть несколько векторов фичей в разный момент времени
features.loc[features.id == 1707210]

,id,buy_time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
2497151,1707210,1539550800,-96.799971,-148.859112,-110.740786,-201.466798,-111.118246,-212.49179,-16.08618,21.363903,-6.78366,56.433462,-2.736081,-4.007526,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-0.37746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,-0.799862,-0.00909,0.648138,0.785634,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,-2942.440404,-1186.765837,-1755.674564,14.368763,-119.724355,-119.662019,-54.602524,-65.059494,128.703718,-7.997875,-10.717958,-28.571103,4.869716,1.911014,-24.171711,-20.331295,-2.958702,-5.745251,-1.671324,-0.001656,-32.681646,-4.882665,-0.265234,-0.408162,-0.091644,-0.237576,-0.295662,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,4.789812,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-2.473446,-5.62084,-1187.961723,-1757.811263,-0.36799,-0.393857,-2945.772987,-2298.725139,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,14.661102,-119.674411,-105.013308,-0.058077,-0.060451,-0.575129,-5.618164,-14.681641,-3.573283,-0.468443,-0.417283,-190.670372,-1.143223,-2.722591,-0.825973,-1.935988,0.0,-1.276187,-0.020137,-0.042636,-29.797016,-116.020802,-42.96363,-34.888325,-3.861461,-0.317164,-0.007024,-0.143269,-43.417671,-0.212646,-0.019562,-0.000044,-0.000379,48.451144,1.738691,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,0.010492,-21.183166,-44.376426,-25.320085,-51.984826,-25.961228,-54.662236,-13.614497,-30.821719,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,72.742637,62.373033,88.067242,66.425238,-30.978121,-65.10072,-16.354808,-35.303704,-0.034569,-0.163184,-109.036398,0.533317,2.070952,-0.623737,-0.228106,0.748041,-0.000567,-0.433736,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,0.665644,0.991001,-3393.574824,-45175.257711,-0.622901,-30.716053,-61790.157098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,-0.622896,-0.00004,-0.620772,-0.012257,-0.107878,-5.726695e+08,-58.544078,-4.405604e+08,-1.204418e+08,8.099332,-1.465191,-33.302382,-265.128986,-37.772492,-0.364694,-0.133771,-0.209468,-33.356505,-109.884564,-317.69102,-5.368281,-235.110707,-108.409742,34.562669,-106.617978,-17.295406,-976.373846,-613.770792,-25.996269,-29.630448,-305.747724,0.167111,-0.694428,-12.175933,-0.45614,1.0
2746322,1707210,1546808400,-351.799971,-230.209112,-245.740786,-162.816798,-251.158246,-173.84179,-101.08618,112.893903,-91.78366,147.963462,-2.736081,-4.007526,-2.558912,-222.019127,-233.34811,-0.000725,-0.016435,-0.107041,-5.41746,-17.348521,106.059185,-95.744164,-0.094251,-0.001733,-0.009327,-2.082209,-0.799862,-0.00909,1.648138,0.785634,0.788392,-0.001884,-0.000023,-0.00003,-87.65939,-0.065583,-14.870765,-192.026959,-2942.440404,-1186.765837,-1755.674564,4

In [6]:
data = pd.read_csv('data_train.csv', sep=',').drop(columns=['Unnamed: 0'])

data.head()  

,id,vas_id,buy_time,target
0,540968,8.0,1537131600,0.0
1,1454121,4.0,1531688400,0.0
2,2458816,1.0,1534107600,0.0
3,3535012,5.0,1535922000,0.0
4,1693214,1.0,1535922000,0.0


In [25]:
data.loc[data.id == 2076220]

,id,vas_id,buy_time,target
377380,2076220,6.0,1542574800,1.0
377378,2076220,4.0,1542574800,1.0
377379,2076220,4.0,1543179600,0.0


In [15]:
data.buy_time.nunique()

26

In [7]:
# объединим data и features в один датафрейм с помощью merge_asof по полю id и выбором ближайшего по buy_time (nearest)
# для этого предварительно нужно отсортировать данные по полю buy_time
data = data.sort_values(by=['buy_time'])
features = features.sort_values(by=['buy_time']) #долго

In [10]:
data_train_merged = pd.merge_asof(data, features, on='buy_time', by='id', direction='nearest')
data_train_merged.head()

,id,vas_id,buy_time,target,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,2582523,2.0,1531083600,0.0,314.560029,9.290888,342.989214,7.523202,337.571754,-13.58179,-16.08618,-65.076097,-6.78366,-30.006538,-2.736081,-4.007526,-2.558912,340.590873,329.26189,-0.000725,-0.016435,-0.107041,-5.41746,1.401479,28.429185,-10.744164,-0.094251,-0.001733,-0.009327,2.497791,0.200138,-0.00909,0.648138,0.785634,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,307.973041,9728.201596,-330.600797,10058.802436,-89.504287,-119.724355,-6.012019,4.514146,-10.526161,160.453718,-4.418641,-10.717958,-28.571103,-2.130284,-4.088986,-30.171711,-25.331295,-1.958702,-6.745251,-1.671324,-0.001656,2.318354,-2.882665,-0.265234,-0.178162,-0.011644,-0.237576,-0.265662,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,1.132417,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-1.473446,-0.62084,-331.796683,10056.665737,0.19201,0.006143,9724.869013,-2298.725139,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,0.091923,0.039549,0.244871,-1.618164,-3.681641,1.426717,0.531557,0.182717,-190.670372,6.856777,11.277409,5.174027,9.064012,0.0,-1.276187,-0.020137,-0.042636,-8.930349,-64.720802,43.03637,-34.888325,-3.861461,0.352836,-0.007024,-0.143269,0.582329,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,0.597843,-0.002595,-1.678214,-0.014542,0.090492,-21.183166,-44.376426,-25.320085,-51.984826,-16.961228,4.454434,-12.614497,-29.955052,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,28.742637,81.589693,50.067242,78.958574,-18.978121,-10.567387,-9.354808,-1.637036,-0.034569,-0.163184,88.758132,-0.466683,2.070952,-0.623737,-0.228106,-0.251959,-0.000567,-0.433736,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,-0.334356,-0.008999,1482.274176,-40034.257711,0.377099,159.323947,-61602.817098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,0.377104,-0.00004,0.379228,-0.012257,-0.107878,9.724973e+08,348.235922,-4.405604e+08,-1.204418e+08,5.613617,-0.465191,-32.302382,193.871014,16.227508,-0.364694,-0.133771,-0.209468,19.643495,-109.884564,854.30898,-4.368281,660.889293,-108.409742,309.562669,508.382022,305.704594,6488.626154,-574.770792,-24.996269,121.369552,142.252276,-16.832889,-0.694428,-11.175933,-0.45614,0.0
1,1292549,2.0,1531083600,0.0,93.880029,-217.499112,79.939214,-270.106798,74.521754,-291.21179,-16.08618,-65.076097,-6.78366,-30.006538,-2.736081,-4.007526,-2.558912,124.490873,113.16189,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,57.973041,-2942.440404,-1186.765837,-1755.674564,-89.504287,-119.724355,-33.478685,-2.55252

In [13]:
data_train_merged.id.nunique() == data.id.nunique() # проверка

True

In [14]:
data_train_merged.buy_time.nunique()

26

In [24]:
data_train_merged.shape

(831653, 257)

In [18]:
# сделаем то же самое для тестовых данных
data_test = pd.read_csv('data_test.csv', sep=',').drop(columns=['Unnamed: 0'])

data_test.head()  

,id,vas_id,buy_time
0,3130519,2.0,1548018000
1,2000860,4.0,1548018000
2,1099444,2.0,1546808400
3,1343255,5.0,1547413200
4,1277040,2.0,1546808400


In [19]:
data_test = test_data.sort_values(by=['buy_time'])

In [20]:
data_test_merged = pd.merge_asof(data_test, features, on='buy_time', by='id', direction='nearest')
data_test_merged.head()

,id,vas_id,buy_time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,2905850,5.0,1546808400,326.930029,227.410888,312.989214,200.223202,307.571754,179.11821,-16.08618,-65.076097,-6.78366,-30.006538,-2.736081,-4.007526,-2.558912,357.540873,346.21189,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,-13.940815,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,2604.991196,-765.412317,3370.403436,-89.504287,-119.724355,-112.262019,-48.269191,-63.992827,-7.896282,49.262935,-10.717958,-28.571103,-28.130284,-33.088986,-59.171711,-26.331295,-4.958702,-9.745251,-1.671324,-0.001656,2.318354,9.117335,0.594766,-0.138162,-0.061644,0.432424,0.104338,-0.028454,-0.044465,-0.301128,-0.554677,-0.036834,-0.130031,-2.783592,-2.60662,-5.390212,-4.022547,0.0,-2.824022,-10.706438,-1.2015,-0.998268,-0.203232,0.0,-0.248755,-0.222852,-0.134088,0.0,-0.030537,-0.125866,-0.096986,-0.679774,-0.626985,-0.691912,-0.506613,-0.185299,-0.598716,-0.000115,-0.250188,-0.348913,-0.828382,-42.275915,-3.950157,-0.253037,-0.318148,-2.29064,-3.447583,-0.040043,-9.408469,-0.212137,-11.955314,-1.019293,-1.473446,-5.62084,-766.608203,3368.266737,0.11201,-0.003857,2601.658613,-1127.119639,-0.343415,-0.08972,-0.278878,-0.433135,-0.024048,-89.211948,-119.674411,-208.886358,0.301923,-0.050451,0.144871,-2.618164,-12.681641,-0.573283,-0.468443,-0.083950,-190.670372,3.856777,1.277409,4.174027,1.064012,0.0,-1.276187,-0.020137,-0.042636,-28.880349,-92.387467,7.03637,-34.888325,-3.861461,-0.317164,-0.007024,-0.143269,-25.417671,-0.212646,-0.019562,-0.000044,-0.000379,-2.548856,-0.261309,-0.536315,-0.061481,-0.152157,-0.002595,-4.678214,-0.014542,-0.359508,-21.183166,-44.376426,-25.320085,-51.984826,-23.961228,-48.328902,-13.614497,-30.821719,-0.028857,-0.063214,-0.019198,-0.033778,-0.003149,-0.005184,-0.001431,-0.00189,-1.257363,-2.793637,-1.932758,-5.008096,-26.978121,-64.034053,-15.354808,-35.153704,-0.034569,-0.163184,-82.947451,-0.466683,-2.929048,0.376263,-0.228106,-0.251959,-0.000567,-0.433736,-0.000708,-0.02921,-0.104665,-0.001358,0.0,0.039208,-0.334356,-0.008999,-11953.712824,-32003.257711,-0.622901,-66.356053,-61855.797098,-0.243136,-42051.166127,-9239.707081,-2.10805,-0.000083,-0.622896,-0.00004,-0.620772,-0.012257,-0.107878,9.643757e+08,365.185922,-4.405604e+08,-1.204418e+08,-8.900668,-1.465191,-33.302382,-208.128986,-39.772492,-0.364694,-0.133771,-0.209468,-35.356505,-109.884564,894.30898,-1.368281,-157.110707,-41.409742,1097.562669,-87.617978,-17.295406,-464.373846,-104.770792,-25.996269,-18.630448,-209.747724,-15.832889,-0.694428,-2.175933,-0.45614,0.0
1,31619,2.0,1546808400,-96.799971,100.290888,-62.040786,250.953202,-67.458246,229.84821,-16.08618,-65.076097,-6.78366,-30.006538,-2.736081,-4.007526,-2.558912,-66.189127,-77.51811,-0.000725,-0.016435,-0.107041,-5.41746,-3.178521,34.759185,-10.744164,-0.094251,-0.001733,-0.009327,-2.082209,0.200138,-0.00909,-0.351862,-0.214366,-0.211608,-0.001884,-0.000023,-0.00003,-2.65939,-0.065583,-0.700765,-192.026959,-2936.883762,-1181.209195,-1755.674564,-89.504287,-119.724355,

In [22]:
data_test_merged.id.count() == data_test.id.count()

True

In [23]:
# сохраним подготовленные данные
data_train_merged.to_pickle('data_train_m.pickle')
data_test_merged.to_pickle('data_test_m.pickle')